In [1]:
import yfinance as yf
import pandas as pd
import numpy as np

# Define a list of stock tickers for screening
tickers = ["AAPL", "MSFT", "NVDA", "TSLA", "AMZN", "GOOGL"]

# Function to fetch data and calculate CANSLIM metrics
def get_canslim_metrics(ticker):
    stock = yf.Ticker(ticker)

    # Fetch financials and history data
    financials = stock.financials
    history = stock.history(period="2y")  # Get last 2 years of price data

    # Calculate the 'C' and 'A' metrics
    quarterly_earnings_growth = (financials.loc['Net Income'].pct_change(periods=1) * 100).iloc[-1]  # YOY Quarterly Earnings Growth
    annual_earnings_growth = (financials.loc['Net Income'].pct_change(periods=4) * 100).iloc[-1]  # YOY Annual Earnings Growth

    # Calculate the 'S' and 'L' metrics
    current_price = history["Close"].iloc[-1]
    year_high = history["Close"].max()
    year_low = history["Close"].min()
    relative_strength = ((current_price - year_low) / (year_high - year_low)) * 100  # Normalize to 100 scale

    # Check Institutional Sponsorship ('I') - High volume implies institutional interest
    avg_volume = history["Volume"].rolling(window=50).mean().iloc[-1]

    return {
        "Ticker": ticker,
        "Quarterly Earnings Growth (%)": quarterly_earnings_growth,
        "Annual Earnings Growth (%)": annual_earnings_growth,
        "Relative Strength (%)": relative_strength,
        "Average Volume (50-day)": avg_volume,
        "Current Price": current_price
    }

# Collect CANSLIM metrics for each ticker
metrics = [get_canslim_metrics(ticker) for ticker in tickers]
canslim_df = pd.DataFrame(metrics)

# Set CANSLIM criteria thresholds
canslim_criteria = {
    "Quarterly Earnings Growth (%)": 20,  # at least 20% growth
    "Annual Earnings Growth (%)": 15,     # at least 15% growth
    "Relative Strength (%)": 80,          # Relative strength in the top 20%
    "Average Volume (50-day)": 5000000,   # Prefer high-volume stocks for institutional support
}

# Filter stocks based on CANSLIM criteria
selected_stocks = canslim_df[
    (canslim_df["Quarterly Earnings Growth (%)"] > canslim_criteria["Quarterly Earnings Growth (%)"]) &
    (canslim_df["Annual Earnings Growth (%)"] > canslim_criteria["Annual Earnings Growth (%)"]) &
    (canslim_df["Relative Strength (%)"] > canslim_criteria["Relative Strength (%)"]) &
    (canslim_df["Average Volume (50-day)"] > canslim_criteria["Average Volume (50-day)"])
]

# Display the selected stocks
print("Selected Stocks based on CANSLIM Criteria:")
print(selected_stocks)

Selected Stocks based on CANSLIM Criteria:
Empty DataFrame
Columns: [Ticker, Quarterly Earnings Growth (%), Annual Earnings Growth (%), Relative Strength (%), Average Volume (50-day), Current Price]
Index: []
